In [23]:
from pathlib import Path

base = Path("/workspaces/micropython-stubs/tests/quality_tests/check_esp32")
if base.exists():
    print(f"Found {base}")


for f in (
    "typings/sys.pyi",
    "typings/stdlib/sys.pyi",
    "typings/stdlib/types.pyi",
    "typings/stdlib/builtins.pyi",
    "typings/stdlib/typing.pyi",
):
    file = base / f
    if file.exists():
        print(f"Removing {f}")
        file.unlink()
    else:
        print(f"File {f} not found")

Found /workspaces/micropython-stubs/tests/quality_tests/check_esp32
File typings/sys.pyi not found
File typings/stdlib/sys.pyi not found
File typings/stdlib/types.pyi not found
File typings/stdlib/builtins.pyi not found
Removing typings/stdlib/typing.pyi


In [25]:

# run mypy
!cd  /workspaces/micropython-stubs/tests/quality_tests/check_esp32
!mypy .

mypy: "tests/quality_tests/feat_uasyncio/typings/stdlib/_collections_abc.pyi" shadows library module "_collections_abc"
note: A user-defined top-level module with name "_collections_abc" is not supported
